In [1]:
from utils.df_handle import *

Default bq project:  spatial-vision-343005.biteam


In [ ]:
import pendulum
from airflow import DAG
from airflow.operators.dummy_operator import DummyOperator
from airflow.operators.python_operator import PythonOperator

local_tz = pendulum.timezone("Asia/Bangkok")

name='PDA_SO'
prefix='SYNC_'
csv_path = '/usr/local/airflow/plugins'+'/'

dag_params = {
    'owner': 'airflow',
    "depends_on_past": False,
    'start_date': datetime(2022, 5, 16, tzinfo=local_tz),
    'email_on_failure': True,
    'email_on_retry': False,
    'email':['duyvq@merapgroup.com', 'vanquangduy10@gmail.com'],
    'do_xcom_push': False,
    'execution_timeout':timedelta(seconds=300)
    # 'retries': 3,
    # 'retry_delay': timedelta(minutes=10),
}

dag = DAG(prefix+name,
          catchup=False,
          default_args=dag_params,
          schedule_interval= '0 2 * * *',
          tags=[prefix+name, 'Daily', '60mins']
)

In [2]:
start_date = '2022-01-01'
datenow = datetime.now().strftime("%Y-%m-%d")
datenow_mns1 = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")
fdom = datetime.now().replace(day=1).strftime("%Y%m%d")
datenow_mns45 = (datetime.now() - timedelta(days=45)).strftime("%Y-%m-%d")

In [5]:
def update_sync_dms_pda_so():
    try:
        table_name = "sync_dms_pda_so"
        from_tb = "OM_PDASalesOrd"
        status_sql = \
        f"""
        select Status, count(*) as count FROM {from_tb}  where CAST(Crtd_DateTime as DATE) >= '{start_date}' group by Status
        """
        countdms = get_ms_df(status_sql)
        # countdms.head()
        print(countdms)
        statusbq_sql = \
        f"""
        SELECT status as Status, count(*) as count from biteam.{table_name} group by status
        """
        countbq = get_bq_df(statusbq_sql)
        print(countbq)
        dfs = dfs_diff(countbq, countdms)
        assert dfs.shape[0] == 0, "NO DIFF"
        print("NO DIFF")
    except AssertionError:
        sql = \
        f"""
        DECLARE @from DATE = '{datenow_mns45}'
        DECLARE @to DATE = '2022-01-31'
        SELECT
        BranchID,
        OrderNbr,
        CustID,
        SlsPerID,
        Status,
        Crtd_Prog,
        Crtd_User,
        Crtd_DateTime,
        LUpd_DateTime,
        LUpd_User,
        Remark,
        InsertFrom,
        BranchRouteID,
        SalesRouteID,
        DeliveryTime,
        OriOrderNbrUp,
        Version
        from {from_tb}
        where Crtd_Datetime >= @from
        """
        df = get_ms_df(sql)
        df['inserted_at'] = datetime.now()
        sql = \
        f"""
        delete from biteam.{table_name} where date(crtd_datetime) >= '{datenow_mns45}'
        """
        print("delete_sql: ", sql)
        execute_bq_query(sql)
        bq_values_insert(df, f"{table_name}", 2)

In [6]:
# update_sync_dms_pda_so()

  Status  count
0      C  83278
1      D     12
2      E   8189
3      H     75
  Status  count
0      E   8189
1      H     75
2      D     12
3      C  83278
NO DIFF


In [5]:
def update_sync_dms_so():
    try:
        table_name = "sync_dms_so"
        from_tb = "OM_SalesOrd"
        status_sql = \
        f"""
        select Status, count(*) as count FROM {from_tb}  where CAST(Crtd_DateTime as DATE) >= '{start_date}' group by Status
        """
        countdms = get_ms_df(status_sql)
        # countdms.head()
        print(countdms)
        statusbq_sql = \
        f"""
        SELECT status as Status, count(*) as count from biteam.{table_name} group by status
        """
        countbq = get_bq_df(statusbq_sql)
        print(countbq)
        dfs = dfs_diff(countbq, countdms)
        assert dfs.shape[0] == 0, "NO DIFF"
        print("NO DIFF")
    except AssertionError:
        pass
        # sql = \
        # f"""
        # DECLARE @from DATE = '{datenow_mns45}'
        # DECLARE @to DATE = '2022-01-31'
        # SELECT
        # BranchID,
        # OrderNbr,
        # ARBatNbr,
        # ARRefNbr,
        # CustID,
        # INBatNbr,
        # INRefNbr,
        # InvcNbr,
        # InvcNote,
        # OrderDate,
        # OrderType,
        # OrigOrderNbr,
        # SlsPerID,
        # Status,
        # Terms,
        # Crtd_Prog,
        # Crtd_User,
        # Crtd_DateTime,
        # LUpd_DateTime,
        # Remark,
        # PaymentsForm,
        # ContractID,
        # InvoiceCustID,
        # SalesOrderType,
        # ReplForOrdNbr,
        # Version,
        # AccumulateAmt
        # from {from_tb}
        # where Crtd_Datetime >= @from
        # """
        # df = get_ms_df(sql)
        # df['inserted_at'] = datetime.now()
        # sql = \
        # f"""
        # delete from biteam.{table_name} where date(crtd_datetime) >= '{datenow_mns45}'
        # """
        # print("delete_sql: ", sql)
        # execute_bq_query(sql)
        # bq_values_insert(df, f"{table_name}", 2)

In [7]:
update_sync_dms_so()

  Status  count
0      C  86412
1      H      1
2      I    435
3      N      1
4      V    254
  Status  count
0      N      1
1      V    254
2      I    438
3      H      1
4      C  86412


In [4]:
def update_sync_dms_ib():
    try:
        table_name = "sync_dms_ib"
        from_tb = "OM_IssueBook"
        status_sql = \
        f"""
        select Status, count(*) as count FROM {from_tb}  where CAST(Crtd_DateTime as DATE) >= '{start_date}' group by Status
        """
        countdms = get_ms_df(status_sql)
        # countdms.head()
        print(countdms)
        statusbq_sql = \
        f"""
        SELECT status as Status, count(*) as count from biteam.{table_name} group by status
        """
        countbq = get_bq_df(statusbq_sql)
        print(countbq)
        dfs = dfs_diff(countbq, countdms)
        assert dfs.shape[0] == 0, "NO DIFF"
        print("NO DIFF")
    except AssertionError:
        sql = \
        f"""
        DECLARE @from DATE = '{datenow_mns45}'
        DECLARE @to DATE = '2022-01-31'
        SELECT
        CONCAT(BranchID,BatNbr) as pk,
        BranchID,
        BatNbr,
        DeliveryUnit,
        SlsperID,
        TruckID,
        Status,
        IssueDate,
        Printed,
        Crtd_DateTime,
        Crtd_Prog,
        Crtd_User,
        LUpd_DateTime
        from {from_tb}
        where Crtd_Datetime >= @from
        """
        df = get_ms_df(sql)
        df['inserted_at'] = datetime.now()
        sql = \
        f"""
        delete from biteam.{table_name} where date(crtd_datetime) >= '{datenow_mns45}'
        """
        print("delete_sql: ", sql)
        execute_bq_query(sql)
        bq_values_insert(df, f"{table_name}", 2)

In [5]:
update_sync_dms_ib()

  Status  count
0      H     44
1      C  19150
  Status  count
0      H    121
1      C  19146
delete_sql:  
        delete from biteam.sync_dms_ib where date(crtd_datetime) >= '2022-04-03'
        


1it [00:04,  4.99s/it]


In [10]:
# update_sync_dms_ib()

  Status  count
0      H     60
1      C  18844
  Status  count
0      H     60
1      C  18844
NO DIFF


In [3]:
def update_sync_dms_dv():
    try:
        table_name = "sync_dms_dv"
        from_tb = "OM_Delivery"
        status_sql = \
        f"""
        select Status, count(*) as count FROM {from_tb}  where CAST(Crtd_DateTime as DATE) >= '{start_date}' group by Status
        """
        countdms = get_ms_df(status_sql)
        # countdms.head()
        print(countdms)
        statusbq_sql = \
        f"""
        SELECT status as Status, count(*) as count from biteam.{table_name} group by status
        """
        countbq = get_bq_df(statusbq_sql)
        print(countbq)
        dfs = dfs_diff(countbq, countdms)
        assert dfs.shape[0] == 0, "NO DIFF"
        print("NO DIFF")
    except AssertionError:
        # pass
        sql = \
        f"""
        DECLARE @from DATE = '{datenow_mns45}'
        DECLARE @to DATE = '2022-01-31'
        SELECT
        BranchID,
        BatNbr,
        OrderNbr,
        Sequence,
        SlsperID,
        Status,
        Crtd_DateTime,
        Crtd_Prog,
        Crtd_User,
        LUpd_DateTime
        from {from_tb}
        where Crtd_Datetime >= @from
        """
        df = get_ms_df(sql)
        df['inserted_at'] = datetime.now()
        sql = \
        f"""
        delete from biteam.{table_name} where date(crtd_datetime) >= '{datenow_mns45}'
        """
        print("delete_sql: ", sql)
        execute_bq_query(sql)
        bq_values_insert(df, f"{table_name}", 2)

In [ ]:
# HANDLE NO STATUS TABLES

In [13]:
def update_sync_dms_pda_sod():
    try:
        table_name = "sync_dms_pda_sod"
        from_tb = "OM_PDASalesOrdDet"
        status_sql = \
        f"""
        select count(*) as count FROM {from_tb}  where CAST(Crtd_DateTime as DATE) >= '{start_date}'
        """
        countdms = get_ms_df(status_sql)
        # countdms.head()
        print(countdms)
        statusbq_sql = \
        f"""
        SELECT count(*) as count from biteam.{table_name}
        """
        countbq = get_bq_df(statusbq_sql)
        print(countbq)
        dfs = dfs_diff(countbq, countdms)
        assert dfs.shape[0] == 0, "NO DIFF"
        print("NO DIFF")
    except AssertionError:
        # pass
        sql = \
        f"""
        DECLARE @from DATE = '{datenow_mns45}'
        DECLARE @to DATE = '2022-01-31'
        CONCAT(BranchID,OrderNbr,LineRef) as pk,
        BranchID,
        OrderNbr,
        LineRef,
        InvtID,
        LineQty,
        OrderType,
        OrigOrderNbr,
        SiteID,
        SlsPrice,
        Crtd_Prog,
        Crtd_User,
        Crtd_Datetime,
        LUpd_Datetime,
        SlsperID,
        BeforeVATPrice,
        BeforeVATAmount,
        AfterVATPrice,
        AfterVATAmount,
        VATAmount
        from {from_tb}
        where Crtd_Datetime >= @from
        """
        df = get_ms_df(sql)
        df['inserted_at'] = datetime.now()
        sql = \
        f"""
        delete from biteam.{table_name} where date(crtd_datetime) >= '{datenow_mns45}'
        """
        print("delete_sql: ", sql)
        execute_bq_query(sql)
        bq_values_insert(df, f"{table_name}", 2)

In [17]:
def update_sync_dms_sod():
    try:
        table_name = "sync_dms_sod1"
        from_tb = "OM_SalesOrdDet"
        status_sql = \
        f"""
        select count(*) as count FROM {from_tb}  where CAST(Crtd_DateTime as DATE) >= '{start_date}'
        """
        countdms = get_ms_df(status_sql)
        # countdms.head()
        print(countdms)
        statusbq_sql = \
        f"""
        SELECT count(*) as count from biteam.{table_name}
        """
        countbq = get_bq_df(statusbq_sql)
        print(countbq)
        dfs = dfs_diff(countbq, countdms)
        assert dfs.shape[0] == 0, "NO DIFF"
        print("NO DIFF")
    except AssertionError:
        # pass
        sql = \
        f"""
        DECLARE @from DATE = '{datenow_mns45}'
        DECLARE @to DATE = '2022-01-31'
        CONCAT(BranchID, OrderNbr, LineRef) as pk,
        BranchID,
        OrderNbr,
        LineRef,
        FreeItem,
        InvtID,
        LineQty,
        OrderType,
        OrigOrderNbr,
        SiteID,
        Crtd_Prog,
        Crtd_User,
        Crtd_Datetime,
        LUpd_Datetime,
        SlsperID,
        OriginalLineRef,
        BeforeVATPrice,
        BeforeVATAmount,
        AfterVATPrice,
        AfterVATAmount,
        VATAmount
        from {from_tb}
        where Crtd_Datetime >= @from
        """
        df = get_ms_df(sql)
        df['inserted_at'] = datetime.now()
        sql = \
        f"""
        delete from biteam.{table_name} where date(crtd_datetime) >= '{datenow_mns45}'
        """
        print("delete_sql: ", sql)
        execute_bq_query(sql)
        bq_values_insert(df, f"{table_name}", 2)

In [ ]:
def update_sync_dms_ibd():
    try:
        table_name = "sync_dms_ibd"
        from_tb = "OM_IssueBookDet"
        status_sql = \
        f"""
        select count(*) as count FROM {from_tb}  where CAST(Crtd_DateTime as DATE) >= '{start_date}'
        """
        countdms = get_ms_df(status_sql)
        # countdms.head()
        print(countdms)
        statusbq_sql = \
        f"""
        SELECT count(*) as count from biteam.{table_name}
        """
        countbq = get_bq_df(statusbq_sql)
        print(countbq)
        dfs = dfs_diff(countbq, countdms)
        assert dfs.shape[0] == 0, "NO DIFF"
        print("NO DIFF")
    except AssertionError:
        # pass
        sql = \
        f"""
        DECLARE @from DATE = '{datenow_mns45}'
        DECLARE @to DATE = '2022-01-31'
        SELECT
        CONCAT(BranchID,BatNbr,OrderNbr) as pk,
        BranchID,
        BatNbr,
        OrderNbr,
        Status,
        DeliveryTime,
        Crtd_DateTime,
        Crtd_Prog,
        Crtd_User,
        LUpd_DateTime,
        Transporters
        from {from_tb}
        where Crtd_Datetime >= @from
        """
        df = get_ms_df(sql)
        df['inserted_at'] = datetime.now()
        sql = \
        f"""
        delete from biteam.{table_name} where date(crtd_datetime) >= '{datenow_mns45}'
        """
        print("delete_sql: ", sql)
        execute_bq_query(sql)
        bq_values_insert(df, f"{table_name}", 2)

In [4]:
def update_sync_dms_iv():
    try:
        table_name = "sync_dms_iv"
        from_tb = "OM_Invoice"
        status_sql = \
        f"""
        select count(*) as count FROM {from_tb}  where CAST(Crtd_DateTime as DATE) >= '{start_date}'
        """
        countdms = get_ms_df(status_sql)
        # countdms.head()
        print(countdms)
        statusbq_sql = \
        f"""
        SELECT count(*) as count from biteam.{table_name}
        """
        countbq = get_bq_df(statusbq_sql)
        print(countbq)
        dfs = dfs_diff(countbq, countdms)
        assert dfs.shape[0] == 0, "NO DIFF"
        print("NO DIFF")
    except AssertionError:
        # pass
        sql = \
        f"""
        DECLARE @from DATE = '{datenow_mns45}'
        DECLARE @to DATE = '2022-01-31'
        SELECT
        CONCAT(BranchID,RefNbr) as pk,
        BranchID,
        RefNbr,
        InvcNbr,
        InvcNote,
        Crtd_DateTime,
        LUpd_User,
        LUpd_DateTime
        from {from_tb}
        where Crtd_Datetime >= @from
        """
        df = get_ms_df(sql)
        df['inserted_at'] = datetime.now()
        sql = \
        f"""
        delete from biteam.{table_name} where date(crtd_datetime) >= '{datenow_mns45}'
        """
        print("delete_sql: ", sql)
        execute_bq_query(sql)
        bq_values_insert(df, f"{table_name}", 2)

In [7]:
def update_sync_dms_err():
    try:
        table_name = "sync_dms_err"
        from_tb = "API_HistoryOM205"
        status_sql = \
        f"""
        select count(*) as count FROM {from_tb}  where CAST(Crtd_DateTime as DATE) >= '{start_date}'
        """
        countdms = get_ms_df(status_sql)
        # countdms.head()
        print(countdms)
        statusbq_sql = \
        f"""
        SELECT count(*) as count from biteam.{table_name}
        """
        countbq = get_bq_df(statusbq_sql)
        print(countbq)
        dfs = dfs_diff(countbq, countdms)
        assert dfs.shape[0] == 0, "NO DIFF"
        print("NO DIFF")
    except AssertionError:
        # pass
        sql = \
        f"""
        DECLARE @from DATE = '{datenow_mns45}'
        DECLARE @to DATE = '2022-01-31'
        DECLARE @StringApproveOrderT1 INT = ( SELECT LEN(N'Đơn Hàng DH032021-00257 Tồn Tại Các Lỗi:</br>') + 1)
        select
        CONCAT(BranchID,OrderNbr) as pk,
        BranchID,
        OrderNbr,
        DateImport as Crtd_DateTime,
        DateUpdate as LUpd_DateTime,
        REPLACE(
        REPLACE
        (SUBSTRING(ErrorMessage, @StringApproveOrderT1, LEN(ErrorMessage)),
        N'</br>Bạn Không Thể Duyệt Đơn.',
        ''),
        '</br>',
        ', '
        )
        AS ErrorMessage
        from {from_tb}
        where Status = 'E'
        and DateImport >= @from
        """
        df = get_ms_df(sql)
        df['inserted_at'] = datetime.now()
        sql = \
        f"""
        delete from biteam.{table_name} where date(crtd_datetime) >= '{datenow_mns45}'
        """
        print("delete_sql: ", sql)
        execute_bq_query(sql)
        bq_values_insert(df, f"{table_name}", 2)

In [8]:
update_sync_dms_err()

DatabaseError: Execution failed on sql '
        select count(*) as count FROM API_HistoryOM205  where CAST(Crtd_DateTime as DATE) >= '2022-01-01'
        ': ('42S22', "[42S22] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Invalid column name 'Crtd_DateTime'. (207) (SQLExecDirectW)")

In [ ]:
dummy_start = DummyOperator(task_id="dummy_start", dag=dag)

update_sync_dms_pda_so = PythonOperator(task_id="update_sync_dms_pda_so", python_callable=update_sync_dms_pda_so, dag=dag)

update_sync_dms_so = PythonOperator(task_id="update_sync_dms_so", python_callable=update_sync_dms_so, dag=dag)

update_sync_dms_ib = PythonOperator(task_id="update_sync_dms_ib", python_callable=update_sync_dms_ib, dag=dag)

update_sync_dms_dv = PythonOperator(task_id="update_sync_dms_dv", python_callable=update_sync_dms_dv, dag=dag)

dummy_start >> [update_sync_dms_pda_so, update_sync_dms_so, update_sync_dms_ib, update_sync_dms_dv]